In [1]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
model = "gpt-4o-mini"

client = OpenAI(api_key=OPENAI_API_KEY)

### 사용 토큰 확인하기

In [2]:

response = client.responses.create(
    model=model,
    input='llm에서 사용하는 토큰에 대해 설명해줘.'
)
print(response.output_text)
print("\n")


print(f"입력 토큰: {response.usage.input_tokens}") # 질문(Prompt)에 사용된 토큰
print(f"출력 토큰: {response.usage.output_tokens}") # 답변(Completion)에 사용된 토큰
print(f"합계 토큰: {response.usage.total_tokens}") # 전체 사용 토큰

LLM(대규모 언어 모델)에서 사용하는 "토큰"은 모델이 텍스트 데이터를 처리하는 기본 단위입니다. 토큰은 단어, 부분 단어, 또는 심지어 문자와 같이 다양한 형태를 가질 수 있습니다. 다음은 토큰의 몇 가지 주요 특징입니다:

1. **정의**: 토큰은 모델이 이해할 수 있는 입력의 단위로, 이는 단어 전체일 수도 있고, 어떤 경우에는 일부분(예: 영어의 'playing'은 'play'와 '-ing'으로 나뉘어질 수 있음)일 수 있습니다.

2. **인코딩**: 자연어 처리에서 텍스트는 토큰화(tokenization) 과정을 거쳐 토큰으로 분리됩니다. 이 과정은 문장에서 단어를 추출하거나 문장 내부의 특정 부분을 분리하는 작업을 포함합니다.

3. **모델 입력**: LLM은 이러한 토큰들을 벡터 형태로 변환하여 모델에 입력합니다. 각 토큰은 고유한 수치적 표현(임베딩)을 갖게 됩니다.

4. **맥락 학습**: 모델은 토큰 간의 관계와 순서를 학습하여 문맥을 이해하고, 문법 및 의미를 추론하는 방식으로 작동합니다.

5. **제한**: LLM은 일반적으로 한 번에 처리할 수 있는 토큰의 수에 제한이 있는데, 이 제한은 모델의 구조와 크기에 따라 다릅니다. 예를 들어, 특정 모델은 최대 2048개의 토큰까지만 처리할 수 있습니다.

6. **효율성**: 토큰화 방식에 따라 입력 데이터의 길이와 모델의 처리 성능에 영향을 미칠 수 있기 때문에, 최적의 토큰화 전략을 선택하는 것이 중요합니다.

결론적으로, 토큰은 LLM이 텍스트를 이해하고 생성하는 데 있어 필수적인 요소로 기능합니다. 효과적인 토큰화는 모델의 성능에 중요한 영향을 미칠 수 있습니다.


입력 토큰: 19
출력 토큰: 437
합계 토큰: 456


### 요청 전 입력 토큰 확인하기

`pip install tiktoken` 를 통해 설치한다.

In [6]:
import tiktoken

def count_tokens(text, model="gpt-4o-mini"):
    # 모델에 맞는 인코딩 가져오기
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        # 모델명을 찾을 수 없을 때 기본값(o200k) 사용
        encoding = tiktoken.get_encoding("o200k_base")
    
    # 텍스트를 토큰으로 변환(encode)
    tokens = encoding.encode(text)
    
    # 토큰 개수 반환
    return len(tokens)

# 테스트
text = "안녕하세요, 이 문장의 토큰은 몇개일까요?"
text = "hello, how many tokens this sentense contains?"
print(f"텍스트: {text}")
print(f"토큰 수: {count_tokens(text)}개")

텍스트: hello, how many tokens this sentense contains?
토큰 수: 10개


### 프롬프트 캐싱
프롬프트 캐싱은 자주 반복되는 프롬프트의 앞부분을 서버에 미리 저장해 두었다가, 동일한 요청이 들어오면 다시 계산하지 않고 즉시 재사용하는 기능이다.  
openai에서는 프롬프트 앞에 1024토큰 단위로 저장하며, 조금이라도 text가 달라지면 재사용하지 못한다.

In [7]:
prompt = """
<content>
Prompt engineering: overview and guide
Last Updated: 01/14/2026

The rise of large language models (LLMs) has brought forth exciting possibilities for human-computer interaction. However, harnessing the full potential of these powerful AI models requires a crucial skill: prompt engineering. This burgeoning field focuses on crafting effective prompts that unlock the capabilities of LLMs, enabling them to understand intent, follow instructions, and generate desired outputs. As we increasingly interact with AI in various applications, prompt engineering plays a vital role in ensuring accurate, relevant, and safe interactions.

prompt engineer

What is prompt engineering?
Prompt engineering is the art and science of designing and optimizing prompts to guide AI models, particularly LLMs, towards generating the desired responses. By carefully crafting prompts, you provide the model with context, instructions, and examples that help it understand your intent and respond in a meaningful way. Think of it as providing a roadmap for the AI, steering it towards the specific output you have in mind.

To dive deeper into the world of prompt design and explore its applications, check out the Introduction to Prompt Design on Google Cloud.

Ready to experiment with LLMs and prompt engineering firsthand? Try the  Vertex AI free trial and experience the power of this technology.

What is a prompt for AI?
In the context of AI, a prompt is the input you provide to the model to elicit a specific response. This can take various forms, ranging from simple questions or keywords to complex instructions, code snippets, or even creative writing samples. The effectiveness of your prompt directly influences the quality and relevance of the AI's output.

What do you need for prompt engineering?
Several key elements contribute to effective prompt engineering. Mastering these allows you to communicate effectively with AI models and unlock their full potential.

Prompt format
The structure and style of your prompt play a significant role in guiding the AI's response. Different models may respond better to specific formats, such as:

The format of your prompt plays a significant role in how the AI interprets your request. Different models may respond better to specific formats, such as natural language questions, direct commands, or structured inputs with specific fields. Understanding the model's capabilities and preferred format is essential for crafting effective prompts.

Context and examples
Providing context and relevant examples within your prompt helps the AI understand the desired task and generate more accurate and relevant outputs. For instance, if you're looking for a creative story, including a few sentences describing the desired tone or theme can significantly improve the results.

Fine-tuning and adapting
Fine-tuning the AI model on specific tasks or domains using tailored prompts can enhance its performance. Additionally, adapting prompts based on user feedback or model outputs can further improve the model's responses over time.

Multi-turn conversations
Designing prompts for multi-turn conversations allows users to engage in continuous and context-aware interactions with the AI model, enhancing the overall user experience.

Types of prompts
There are various types of prompts used in AI, each serving a specific purpose:

Direct prompts (Zero-shot)
Zero-shot prompting involves providing the model with a direct instruction or question without any additional context or examples. 

An example of this is idea generation, where the model is prompted to generate creative ideas or brainstorming solutions. Another example is summarization, or translation, where the model is asked to summarize or translate some piece of content.

One-, few- and multi-shot prompts
This method involves providing the model with one or more examples of the desired input-output pairs before presenting the actual prompt. This can help the model better understand the task and generate more accurate responses.

Chain of Thought Prompts
CoT prompting encourages the model to break down complex reasoning into a series of intermediate steps, leading to a more comprehensive and well-structured final output.

Zero-shot CoT Prompts
Combines chain of thought prompting with zero-shot prompting by asking the model to perform reasoning steps, which may often produce better output.

Use cases and examples of prompt engineering
Here are some specific examples and use cases showing how prompt engineering helps produce customized and relevant output.

Language and Text Generation
Scenario

Instructions

Example Prompt

Creative Writing

Craft prompts that specify genre, tone, style, and plot points to guide the AI in generating engaging narratives.

"Write a short story about a young woman who discovers a magical portal in her attic."

Summarization

Provide the AI with text and instruct it to generate concise summaries that capture key information.

“Summarize the main points of the following news article on climate change."

Translation

Specify the source and target languages to enable the AI to accurately translate text while preserving meaning and context.

"Translate the following text from English to Spanish: 'The quick brown fox jumps over the lazy dog.'"

Dialogue

Design prompts that simulate conversations, allowing the AI to generate responses that mimic human interaction and maintain context.

"You are a friendly chatbot helping users troubleshoot their computer problems. Respond to the user's query: 'My computer won't turn on.'"

Question Answering
Scenario

Instructions

Example Prompt

Open-Ended Questions

Formulate prompts that encourage the AI to provide comprehensive and informative answers based on its knowledge base.

"Explain the concept of quantum computing and its potential impact on the future of technology."

Specific Questions

Design prompts that target specific information, enabling the AI to retrieve precise answers from the provided context or its internal knowledge base.

"What is the capital of France?" or "According to the provided text, what are the main causes of deforestation?"

Multiple Choice Questions

Present prompts with options, prompting the AI to analyze and select the most appropriate answer based on its understanding of the context.

"Who wrote the Harry Potter series? A) J.R.R. Tolkien, B) J.K. Rowling, C) Stephen King"

Hypothetical Questions

Craft prompts that explore hypothetical situations, allowing the AI to reason, speculate, and provide potential outcomes or solutions.

"What would happen if humans could travel at the speed of light?"

Opinion-Based Questions

Design prompts that elicit the AI's perspective or opinion on a specific topic, encouraging it to provide reasoning and justification for its stance.

"Do you believe that artificial intelligence will eventually surpass human intelligence? Why or why not?"

Code Generation
Scenario

Instructions

Example Prompt

Code Completion

Provide the AI with a partial code snippet and prompt it to suggest or complete the remaining code based on the context and programming language.

"Write a Python function to calculate the factorial of a given number."

Code Translation

Specify the source and target programming languages to enable the AI to translate code while preserving functionality and syntax.

"Translate the following Python code to JavaScript: def greet(name): print('Hello,', name)"

Code Optimization

Prompt the AI to analyze existing code and suggest improvements for efficiency, readability, or performance.

"Optimize the following Python code to reduce its execution time."

Code Debugging

Provide the AI with code containing errors and prompt it to identify and suggest potential solutions for the identified issues.

"Debug the following Java code and explain why it is throwing a NullPointerException."


Image Generation
Scenario

Instructions

Example Prompt

Photorealistic Images

Craft prompts that describe the desired image in detail, including objects, scenery, lighting, and style, to generate realistic and high-quality images.

"A photorealistic image of a sunset over the ocean with palm trees silhouetted against the sky."

Artistic Images

Design prompts that specify art styles, techniques, and subject matter to guide the AI in creating images that mimic specific artistic movements or evoke certain emotions.

"An impressionist painting of a bustling city street with people walking under umbrellas in the rain."

Abstract Images

Formulate prompts that encourage the AI to generate images that are open to interpretation, utilizing shapes, colors, and textures to evoke feelings or concepts.

"An abstract image representing the concept of hope, using bright colors and flowing shapes."

Image Editing

Provide the AI with an existing image and specify desired modifications, enabling it to edit and enhance the image according to the given instructions.

"Change the background of this photo to a starry night sky and add a full moon." or "Remove the person from this image and replace them with a cat."

Strategies for writing better prompts
Developing effective prompts requires a strategic approach. Consider these strategies to enhance your prompt engineering skills:

1. Set Clear Goals and Objectives:
Tactic

Prompt Example

Use action verbs to specify the desired action

"Write a bulleted list that summarizes the key findings of the attached research paper"

Define the desired length and format of the output

"Compose a 500-word essay discussing the impact of climate change on coastal communities."

Specify the target audience

"Write a product description for a new line of organic skincare products, targeting young adults concerned with sustainability."

2. Provide Context and Background Information:
Tactic

Prompt Example

Include relevant facts and data

"Given that global temperatures have risen by 1 degree Celsius since the pre-industrial era, discuss the potential consequences for sea level rise."

Reference specific sources or documents

"Based on the attached financial report, analyze the company's profitability over the past five years."

Define key terms and concepts

"Explain the concept of quantum computing in simple terms, suitable for a non-technical audience."

3. Use Few-Shot Prompting:
Tactic

Prompt Example

Provide a few examples of desired input-output pairs

Input: "Cat" Output: "A small furry mammal with whiskers." Input: "Dog" Output: "A domesticated canine known for its loyalty." Prompt: "Elephant"

Demonstrate the desired style or tone

Example 1 (humorous): "The politician's speech was so dull, it could cure insomnia." Example 2 (formal): "The dignitary delivered an address that was both informative and engaging." Prompt: "Write a sentence describing the comedian's stand-up routine."

Show the desired level of detail

Example 1 (brief): "The movie was about a young boy who befriends an alien." Example 2 (detailed): "The science fiction film follows the story of Elliot, a lonely boy who discovers and forms a unique bond with an extraterrestrial stranded on Earth." Prompt: "Summarize the plot of the novel you just finished reading."

4. Be Specific:
Tactic

Prompt Example

Use precise language and avoid ambiguity

Instead of: "Write something about climate change," use: "Write a persuasive essay arguing for the implementation of stricter carbon emission regulations."

Quantify your requests whenever possible

Instead of: "Write a long poem," use: "Write a sonnet with 14 lines that explores themes of love and loss."

Break down complex tasks into smaller steps

Instead of: "Create a marketing plan," use: "1. Identify the target audience. 2. Develop key marketing messages. 3. Choose appropriate marketing channels."

5. Iterate and Experiment:
Tactic

Action

Try different phrasings and keywords

Rephrase your prompt using synonyms or alternative sentence structures.

Adjust the level of detail and specificity

Add or remove information to fine-tune the output.

Test different prompt lengths

Experiment with both shorter and longer prompts to find the optimal balance.

6. Leverage Chain of Thought Prompting:
Tactic

Prompt Example

Encourage step-by-step reasoning

"Solve this problem step-by-step: John has 5 apples, he eats 2. How many apples does he have left? Step 1: John starts with 5 apples. Step 2: He eats 2 apples, so we need to subtract 2 from 5. Step 3: 5 - 2 = 3. Answer: John has 3 apples left."

Ask the model to explain its reasoning process

"Explain your thought process in determining the sentiment of this movie review: 'The acting was superb, but the plot was predictable.'"

Guide the model through a logical sequence of thought

"To classify this email as spam or not spam, consider the following: 1. Is the sender known? 2. Does the subject line contain suspicious keywords? 3. Is the email offering something too good to be true?"

For further guidance on prompt engineering best practices, explore the Five Best Practices for Prompt Engineering on Google Cloud.

Benefits of prompt engineering
Effective prompt engineering offers numerous benefits, enhancing the capabilities and usability of AI models:

Improved model performance
Well-crafted prompts lead to more accurate, relevant, and informative outputs from AI models, as they provide clear instructions and context.

Reduced bias and harmful responses
By carefully controlling the input and guiding the AI's focus, prompt engineering helps mitigate bias and minimize the risk of generating inappropriate or offensive content.

Increased control and predictability
Prompt engineering empowers you to influence the AI's behavior and ensure consistent and predictable responses aligned with your desired outcomes.

Enhanced user experience
Clear and concise prompts make it easier for users to interact effectively with AI models, leading to more intuitive and satisfying experiences.
</content>

<instruction>
위의 <content>를 기반으로 다음 요청 사항을 수행해줘.

[요청 사항]
{user_input}
</instruction>
"""



In [9]:
response = client.responses.create(
    model='gpt-4o',
    input=prompt.format(user_input="요약해줘")
)

print(response.usage)
# print(response.output_text)


ResponseUsage(input_tokens=2672, input_tokens_details=InputTokensDetails(cached_tokens=1920), output_tokens=141, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=2813)


In [ ]:
response = client.responses.create(
    model='gpt-4o',
    input=prompt.format(user_input="설명해줘")
)

print(response.usage)
# print(response.output_text)

ResponseUsage(input_tokens=2672, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=697, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=3369)


### LLM에서의 도구 사용
Responses API는 여러 내장 도구들을 가져, text생성 뿐만 아니라 외부 도구와 소통하여 여러 동작들을 할 수 있다.  
web_search, file_search 등 여러 내장 도구들을 활용할 수 있다.

In [13]:
# 일반적으로 실시간 정보에 접근이 불가능하다.

response = client.responses.create(
    model=model,
    input="오늘 뉴스 알려줘"
)

print(response.output_text)

죄송하지만, 실시간 뉴스나 업데이트를 제공할 수는 없습니다. 대신 최근 소식이나 관심 있는 주제에 대해 설명해드릴 수 있습니다. 어떤 분야의 뉴스를 원하시는지 말씀해 주시면, 그에 관련된 정보를 제공해드릴게요.


In [14]:
# web_search tool을 활용하여 웹검색을 통해 실시간 정보에 접근이 가능하다.

response = client.responses.create(
    model=model,
    tools=[{"type": "web_search"}],
    input="오늘 뉴스 알려줘"
)

print(response.output_text)

안녕하세요. 2026년 1월 21일 수요일의 주요 뉴스를 알려드리겠습니다.

**국내 뉴스**

- **'나는 SOLO' 22기 옥순♥경수, 아들 공개**: ENA와 SBS Plus의 예능 프로그램 '나는 SOLO, 그 후 사랑은 계속된다'에서 22기 출연자 옥순과 경수가 아들과 함께한 일상을 공개했습니다. ([v.daum.net](https://v.daum.net/v/20250214101103736?utm_source=openai))

- **'나솔사계' 24기 영식, 옥순에게 전화**: '나는 SOLO, 그 후 사랑은 계속된다'에서 24기 출연자 영식이 데이트 선택 0표의 충격을 받은 후 24기 옥순에게 전화를 걸어 감정을 털어놓는 장면이 방송되었습니다. ([bntnews.co.kr](https://www.bntnews.co.kr/article/view/bnt202511210017?utm_source=openai))

**국제 뉴스**

- **타글레 추기경, 아제르바이잔 가톨릭 공동체 방문**: 교황청 복음화부 부장관인 루이스 안토니오 타글레 추기경이 아제르바이잔을 방문하여 바쿠에 건설 중인 성 요한 바오로 2세 성당 공사 현장을 둘러보고, 현지 가톨릭 공동체와 만남을 가졌습니다. ([vaticannews.va](https://www.vaticannews.va/ko/vatican-city/news/2025-10/tagle-azerbaijan-prefecture.html?utm_source=openai))

- **교황, 건강 상태 약간 호전되며 편안히 수면**: 프란치스코 교황은 최근 건강 상태가 약간 호전되었으며, 밤새 편안히 잠을 자고 오전에도 계속 휴식을 취하고 있다고 교황청 공보실이 밝혔습니다. ([vaticannews.va](https://www.vaticannews.va/ko/pope/news/2025-02/pope-francis-morning-health-update-press-office.html?utm_source=openai))

**미